In [1]:
import numpy as np
import pandas as pd
from scipy import special
from random import sample
import itertools
from datetime import timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math

from dateutil.relativedelta import *
from datetime import date
from datetime import datetime


def formatarStr (DataFrame, columns):
    for col in columns:
        DataFrame[col] = DataFrame[col].str.strip()
        
def calcularIdade(dtNasc):
    try:
        if dtNasc == 0: 
            return np.nan
        else: 
            today = date.today()
            dob = datetime.strptime(str(dtNasc), '%d/%m/%Y').date()
            age = relativedelta(today, dob)
            return age.years
    except (RuntimeError, TypeError, NameError, ValueError):
        return np.nan

def calcularAnoMes(dtAcid): 
    if len(dtAcid) == 10:
        anoMesStr = dtAcid[6:]+dtAcid[3:5]
    elif len(dtAcid) == 7:
        anoMesStr = dtAcid.replace('/', '')
    return anoMesStr

In [2]:
#nomeArquivoOrigem = 'datasets/origem/cat2018-comp01-02-03-2019.csv'
#nomeArquivoResultado2019 = 'datasets/result/dados_cat_201901.csv'
#nomeArquivoResultado2018 = 'datasets/result/dados_cat_2018A.csv'

#nomeArquivoOrigem = 'datasets/origem/cat-comp04-05-06-2019.csv'
#nomeArquivoResultado2019 = 'datasets/result/dados_cat_201902.csv'
#nomeArquivoResultado2018 = 'datasets/result/dados_cat_2018B.csv'

#nomeArquivoOrigem = 'datasets/origem/cat-comp07-08-09-2019.csv'
#nomeArquivoResultado2019 = 'datasets/result/dados_cat_201903.csv'
#nomeArquivoResultado2018 = 'datasets/result/dados_cat_2018C.csv'

nomeArquivoOrigem = 'datasets/origem/cat-comp10-11-12-2019.csv'
nomeArquivoResultado2019 = 'datasets/result/dados_cat_201904.csv'
nomeArquivoResultado2018 = 'datasets/result/dados_cat_2018D.csv'


columns_types = {'agente': str, 'dt_acidente': str, 'cbo': int, 'cbo_desc': str, 'cid': str, 'cid_desc': str, 
                 'cnae': int, 'cnae_desc': str, 'emitente': str, 'especie': str, 'filiacao': str, 
                 'obito': str, 'munic_emp': str, 'natureza': str, 'origem': str, 'parte': str, 'sexo': str, 
                 'tipo_acidente': str, 'uf_acidente': str, 'uf_emp': str, 'dt_afast': str, 'dt_despacho': str, 'dt_acid': str, 
                 'dt_nasc': str, 'dt_emissao': str}

nomes  = ['agente', 'dt_acidente', 'cbo', 'cbo_desc', 'cid', 'cid_desc', 'cnae', 'cnae_desc', 'emitente', 'especie', 
          'filiacao', 'obito', 'munic_emp', 'natureza', 'origem', 'parte', 'sexo', 'tipo_acidente', 'uf_acidente', 'uf_emp', 
          'dt_afast', 'dt_despacho', 'dt_acid', 'dt_nasc', 'dt_emissao']


dados_originais = pd.read_csv(nomeArquivoOrigem, sep = ';', header=0, names=nomes, dtype=columns_types, encoding='iso-8859-1')

df_original = pd.DataFrame(data = dados_originais, columns=nomes)
df_original.shape[0] #117439

113433

In [3]:
df_original.head()

,agente,dt_acidente,cbo,cbo_desc,cid,cid_desc,cnae,cnae_desc,emitente,especie,...,parte,sexo,tipo_acidente,uf_acidente,uf_emp,dt_afast,dt_despacho,dt_acid,dt_nasc,dt_emissao
0,Esforco Excessivo ao,2019/02,784205,784205-Alimentador d,M659,M65.9 Sinovite e Ten,1012,"Abate de Suinos, Ave",Empregador,Pa,...,"Membros Superiores,",Feminino,Doença,{ñ class},Santa Catarina,2019/10,0000/00,01/02/2019,17/09/1973,18/10/2019
1,Chao - Superficie Ut,2019/02,911305,911305-Mec. Manut. M,M170,M17.0 Gonartrose Pri,2391,Aparelhamento e Outr,Empregador,Pa,...,Joelho,Masculino,Típico,{ñ class},Espírito Santo,2019/10,0000/00,07/02/2019,14/12/1971,28/10/2019
2,"Politriz, Lixadora,",2019/01,724410,724410-Caldeireiro (,S660,S66.0 Traum Musc Fle,2829,Fabricacao de Maquin,Empregador,Afastamento Até 15 D,...,Mao (Exceto Punho ou,Masculino,Típico,Maranhão,São Paulo,2019/09,0000/00,31/01/2019,05/04/1974,02/10/2019
3,Rua e Estrada - Supe,2019/02,622110,622110-Trab. Cultura,M511,M51.1 Transt Disco L,1071,Fabricacao de Acucar,Empregador,Pa,...,Dorso (Inclusive Mus,Masculino,Típico,Maranhão,São Paulo,2019/10,0000/00,01/02/2019,06/03/1986,30/10/2019
4,Reacao do Corpo a Mo,2019/02,413210,413210-Caixa Banco,M751,M75.1 Sindr do Mangu,6422,"Bancos Multiplos, co",Sindicato,Pa,...,"Membros Superiores,",Masculino,Doença,Tocantins,Rio de Janeiro,2019/10,0000/00,12/02/2019,07/07/1969,22/10/2019


In [4]:
df_cat = pd.DataFrame(df_original[['agente', 'dt_acidente', 'cbo', 'cid', 'cnae', 'obito', 'natureza', 'parte','sexo', 'tipo_acidente', 
                   'uf_emp', 'dt_nasc']])

formatarStr(df_cat, ['agente', 'dt_acidente', 'cid', 'obito', 'natureza', 'parte', 'sexo', 'tipo_acidente', 'uf_emp', 
                     'dt_nasc'])
df_cat.head(10)

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc
0,Esforco Excessivo ao,2019/02,784205,M659,1012,Não,Inflamacao de Articu,"Membros Superiores,",Feminino,Doença,Santa Catarina,17/09/1973
1,Chao - Superficie Ut,2019/02,911305,M170,2391,Não,Luxacao,Joelho,Masculino,Típico,Espírito Santo,14/12/1971
2,"Politriz, Lixadora,",2019/01,724410,S660,2829,Não,"Corte, Laceracao, Fe",Mao (Exceto Punho ou,Masculino,Típico,São Paulo,05/04/1974
3,Rua e Estrada - Supe,2019/02,622110,M511,1071,Não,"Contusao, Esmagament",Dorso (Inclusive Mus,Masculino,Típico,São Paulo,06/03/1986
4,Reacao do Corpo a Mo,2019/02,413210,M751,6422,Não,"Doenca, Nic","Membros Superiores,",Masculino,Doença,Rio de Janeiro,07/07/1969
5,Piso de Veiculo - Su,2019/02,784205,S934,1062,Não,"Distensao, Torcao",Articulacao do Torno,Masculino,Típico,Rio Grande do Sul,07/04/1993
6,"Esforco Excessivo, N",2019/01,513505,M654,4639,Não,"Doenca, Nic","Membros Superiores,",Masculino,Doença,Pernambuco,29/06/1980
7,Veiculo Rodoviario M,2019/01,911305,S93,2910,Não,"Distensao, Torcao",Ombro,Masculino,Trajeto,Paraná,31/07/1980
8,"Vidraria, Fibra de V",2019/02,300305,S660,3600,Não,"Corte, Laceracao, Fe",Braco (Entre O Punho,Masculino,Típico,Minas Gerais,29/12/1975
9,Prensa - Maquina,2019/02,764105,S610,1529,Não,"Contusao, Esmagament",Dedo,Masculino,Típico,Rio Grande do Sul,24/01/2000


In [5]:
print('Dados ausentes, por coluna: ')
# verificar se, nas colunas mais importantes, há registros com o valor {ñ class}:
f_agente = df_cat[df_cat['agente'] == '{ñ class}'].shape[0] #
print('agente: {}'.format(f_agente))
f_cbo = df_cat[df_cat['cbo'] == '{ñ class}'].shape[0]  #
print('cbo: {}'.format(f_cbo))
f_cid = df_cat[df_cat['cid'] == '{ñ class}'].shape[0]  #
print('cid: {}'.format(f_cid))
f_cnae = df_cat[df_cat['cnae'] == '{ñ class}'].shape[0]  #
print('cnae: {}'.format(f_cnae))
f_obito = df_cat[df_cat['obito'] == '{ñ class}'].shape[0]  #
print('obito: {}'.format(f_obito))
f_natureza = df_cat[df_cat['natureza'] == '{ñ class}'].shape[0]  #
print('natureza: {}'.format(f_natureza))
f_parte = df_cat[df_cat['parte'] == '{ñ class}'].shape[0]  # 
print('parte: {}'.format(f_parte))
f_sexo = df_cat[df_cat['sexo'] == '{ñ class}'].shape[0]  #
print('sexo: {}'.format(f_sexo))
f_tipo = df_cat[df_cat['tipo_acidente'] == '{ñ class}'].shape[0]  #
print('tipo: {}'.format(f_tipo))
f_uf = df_cat[df_cat['uf_emp'] == '{ñ class}'].shape[0]  #
print('uf: {}'.format(f_uf))

#se necessário, apagar as linhas cujo valor de algum campo importante seja nulo
df_cat.drop(df_cat[df_cat['parte'] == '{ñ class}'].index, inplace = True)

df_cat.shape[0] #117169

df_cat.info()

Dados ausentes, por coluna: 
agente: 991
cbo: 0
cid: 0
cnae: 0
obito: 0
natureza: 200
parte: 32
sexo: 0
tipo: 0
uf: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 113401 entries, 0 to 113432
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   agente         113401 non-null  object
 1   dt_acidente    113401 non-null  object
 2   cbo            113401 non-null  int32 
 3   cid            113401 non-null  object
 4   cnae           113401 non-null  int32 
 5   obito          113401 non-null  object
 6   natureza       113401 non-null  object
 7   parte          113401 non-null  object
 8   sexo           113401 non-null  object
 9   tipo_acidente  113401 non-null  object
 10  uf_emp         113401 non-null  object
 11  dt_nasc        113401 non-null  object
dtypes: int32(2), object(10)
memory usage: 10.4+ MB


In [6]:
#tratamento dos valores nulos de 'dt_nasc':
df_cat.fillna(0, inplace = True)
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113401 entries, 0 to 113432
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   agente         113401 non-null  object
 1   dt_acidente    113401 non-null  object
 2   cbo            113401 non-null  int32 
 3   cid            113401 non-null  object
 4   cnae           113401 non-null  int32 
 5   obito          113401 non-null  object
 6   natureza       113401 non-null  object
 7   parte          113401 non-null  object
 8   sexo           113401 non-null  object
 9   tipo_acidente  113401 non-null  object
 10  uf_emp         113401 non-null  object
 11  dt_nasc        113401 non-null  object
dtypes: int32(2), object(10)
memory usage: 10.4+ MB


In [7]:
df_cat['idade'] = df_cat['dt_nasc'].apply(lambda x: calcularIdade(x))
df_cat.head(5)

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
0,Esforco Excessivo ao,2019/02,784205,M659,1012,Não,Inflamacao de Articu,"Membros Superiores,",Feminino,Doença,Santa Catarina,17/09/1973,47.0
1,Chao - Superficie Ut,2019/02,911305,M170,2391,Não,Luxacao,Joelho,Masculino,Típico,Espírito Santo,14/12/1971,49.0
2,"Politriz, Lixadora,",2019/01,724410,S660,2829,Não,"Corte, Laceracao, Fe",Mao (Exceto Punho ou,Masculino,Típico,São Paulo,05/04/1974,46.0
3,Rua e Estrada - Supe,2019/02,622110,M511,1071,Não,"Contusao, Esmagament",Dorso (Inclusive Mus,Masculino,Típico,São Paulo,06/03/1986,35.0
4,Reacao do Corpo a Mo,2019/02,413210,M751,6422,Não,"Doenca, Nic","Membros Superiores,",Masculino,Doença,Rio de Janeiro,07/07/1969,51.0


In [8]:
df_cat[df_cat['idade'].isnull()].shape[0]  
# 81 campos faltantes na coluna, por não haver dt_nasc

32

In [9]:
df_cat[df_cat['idade'].isnull()].head(5)
# 1158
# 2681

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
62754,Calcada ou Caminho p,2019/11,715205,S621,4120,Não,Fratura,Dedo,Não Informado,Típico,Paraná,00/00/0000,NaN
65602,Agua - Usar Quando O,2019/11,514320,T201,8520,Não,Queimadura ou Escald,Torax (Inclusive Org,Não Informado,Típico,São Paulo,00/00/0000,NaN
65744,"Faca, Facao- Ferrame",2019/11,519910,S680,8122,Não,Amputacao ou Enuclea,Dedo,Não Informado,Típico,São Paulo,00/00/0000,NaN
69849,"Ser Vivo, Nic",2019/11,421105,F430,5310,Não,"Doenca, Nic",Sistema Nervoso,Não Informado,Típico,Maranhão,00/00/0000,NaN
76398,"Faca, Facao- Ferrame",2019/11,848510,S600,4711,Não,"Corte, Laceracao, Fe",Dedo,Não Informado,Típico,São Paulo,00/00/0000,NaN


In [10]:
mediaIdades = math.floor(df_cat['idade'].mean())
mediaIdades

37

In [11]:
df_cat.fillna(mediaIdades, inplace=True)

df_cat.isnull().sum()

agente           0
dt_acidente      0
cbo              0
cid              0
cnae             0
obito            0
natureza         0
parte            0
sexo             0
tipo_acidente    0
uf_emp           0
dt_nasc          0
idade            0
dtype: int64

In [12]:
df_cat.columns.name = 'caracteristica'
df_cat.index.name = "id"
df_cat[df_cat.index == 1158]

caracteristica,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
id,,,,,,,,,,,,,
1158,"Ferramenta, Maquina,",2019/10,513430,S500,5620,Não,"Contusao, Esmagament",Antebraco (Entre O P,Feminino,Típico,São Paulo,02/07/1992,28.0


In [13]:
#criar coluna anoMes e deletar a dt_acidente
df_cat['anoMes'] = df_cat['dt_acidente'].apply(lambda x: calcularAnoMes(x))
df_cat.head()

caracteristica,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade,anoMes
id,,,,,,,,,,,,,,
0,Esforco Excessivo ao,2019/02,784205,M659,1012,Não,Inflamacao de Articu,"Membros Superiores,",Feminino,Doença,Santa Catarina,17/09/1973,47.0,201902
1,Chao - Superficie Ut,2019/02,911305,M170,2391,Não,Luxacao,Joelho,Masculino,Típico,Espírito Santo,14/12/1971,49.0,201902
2,"Politriz, Lixadora,",2019/01,724410,S660,2829,Não,"Corte, Laceracao, Fe",Mao (Exceto Punho ou,Masculino,Típico,São Paulo,05/04/1974,46.0,201901
3,Rua e Estrada - Supe,2019/02,622110,M511,1071,Não,"Contusao, Esmagament",Dorso (Inclusive Mus,Masculino,Típico,São Paulo,06/03/1986,35.0,201902
4,Reacao do Corpo a Mo,2019/02,413210,M751,6422,Não,"Doenca, Nic","Membros Superiores,",Masculino,Doença,Rio de Janeiro,07/07/1969,51.0,201902


In [14]:
df_cat.drop(['dt_acidente', 'dt_nasc'], axis=1, inplace = True)
df_cat.head()

caracteristica,agente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,idade,anoMes
id,,,,,,,,,,,,
0,Esforco Excessivo ao,784205,M659,1012,Não,Inflamacao de Articu,"Membros Superiores,",Feminino,Doença,Santa Catarina,47.0,201902
1,Chao - Superficie Ut,911305,M170,2391,Não,Luxacao,Joelho,Masculino,Típico,Espírito Santo,49.0,201902
2,"Politriz, Lixadora,",724410,S660,2829,Não,"Corte, Laceracao, Fe",Mao (Exceto Punho ou,Masculino,Típico,São Paulo,46.0,201901
3,Rua e Estrada - Supe,622110,M511,1071,Não,"Contusao, Esmagament",Dorso (Inclusive Mus,Masculino,Típico,São Paulo,35.0,201902
4,Reacao do Corpo a Mo,413210,M751,6422,Não,"Doenca, Nic","Membros Superiores,",Masculino,Doença,Rio de Janeiro,51.0,201902


In [15]:
df_cat.shape[0]

113401

In [16]:
#apagando os acidentes cujo anoMes não foi preenchido corretamente
condicao = df_cat['anoMes'].str.match('^[0-9]+$') == False
indexItensInvalidos = df_cat[condicao].index

df_cat.drop(indexItensInvalidos, inplace=True)
df_cat.shape[0] #

113401

In [17]:
#apagando as linhas de cujo campo 'cnae' foi preenchido com um código cnae inválido (0)
condicao_cnae_invalido = df_cat['cnae'] == 0
df_cat.drop(df_cat[condicao_cnae_invalido].index, inplace = True)
df_cat.shape[0] #

112869

In [18]:
#separando os acidentes que aconteceram em 2019 dos que aconteceram em 2018:
condicao2019 = df_cat['anoMes'].str.match('^2019[0-9]+') == True
df_cat2019 = df_cat[condicao2019].copy()
df_cat2019.shape[0] #

condicao2018 = df_cat['anoMes'].str.match('^2018[0-9]+') == True
df_cat2018 = df_cat[condicao2018].copy()
df_cat2018.shape[0] #

0

In [19]:
df_cat2019.to_csv(nomeArquivoResultado2019, sep = ';', index=False, encoding='utf-8')

In [20]:
if(df_cat2018.shape[0] > 0):
    df_cat2018.to_csv(nomeArquivoResultado2018, sep = ';', index=False, encoding='UTF-8')